# Análise Exploratória de Dados — Empresas Cadastradas no Programa

Este notebook apresenta uma análise exploratória da base de empresas cadastradas, organizada em cinco seções:

1. Carregamento e limpeza dos dados
2. Visão geral da base
3. Recorte: agronegócio
4. Recorte: Ribeirão Preto
5. Recorte: Pesquisa e Desenvolvimento (CNAE 72)

Para leitores sem familiaridade com código, os blocos cinzas podem ser ignorados. O foco está nos textos explicativos e nos gráficos.

## 1. Carregamento e Limpeza dos Dados

A planilha original continha colunas desnecessárias para esta análise, incluindo dados de contato que foram removidos por boas práticas de privacidade, em uma cópia de trabalho, antes do início do processamento.

Após selecionar as colunas relevantes, foram identificados quatro problemas de qualidade que precisaram ser corrigidos antes de qualquer análise:

- **CNAE** preenchido em formatos variados — com pontos, traços ou combinações diferentes
- **Faturamento** registrado como texto com "R$", e valores zero tratados como dado não informado
- **Estado** com nomes por extenso, siglas e até países estrangeiros
- **Ano de fundação** com mistura de tipos — algumas células foram interpretadas pelo Excel como data completa em vez de número inteiro

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
import seaborn as sns
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# Carregamento e renomeação das colunas
df = pd.read_excel('empresas_dna.xlsx')
df.columns = ['nome', 'ano_fundacao', 'cnae', 'cidade', 'estado', 'descricao', 'faturamento']

# CNAE: remove pontos, traços e outros caracteres não numéricos
# Os 2 primeiros dígitos identificam a divisão de atividade econômica
df['cnae_limpo']   = df['cnae'].astype(str).apply(lambda x: re.sub(r'\D', '', x))
df['cnae_divisao'] = df['cnae_limpo'].str[:2]

# Faturamento: converte texto formatado em número, trata zeros como ausente
def limpar_faturamento(valor):
    if pd.isna(valor):
        return None
    texto = str(valor).replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
    try:
        numero = float(texto)
        return numero if numero > 0 else None
    except:
        return None

df['faturamento_num'] = df['faturamento'].apply(limpar_faturamento)

# Estado: extrai sigla entre parênteses, mapeia nomes por extenso
# Entradas não reconhecidas são classificadas como Exterior
SIGLAS_BR = {
    'AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS',
    'MG','PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC','SP','SE','TO'
}
NOMES_PARA_SIGLA = {
    'SAO PAULO': 'SP', 'SÃO PAULO': 'SP', 'MINAS GERAIS': 'MG',
    'RIO DE JANEIRO': 'RJ', 'PARANA': 'PR', 'PARANÁ': 'PR',
    'SANTA CATARINA': 'SC', 'RIO GRANDE DO SUL': 'RS', 'BAHIA': 'BA',
    'GOIAS': 'GO', 'GOIÁS': 'GO', 'MATO GROSSO': 'MT',
    'MATO GROSSO DO SUL': 'MS', 'PERNAMBUCO': 'PE', 'CEARA': 'CE', 'CEARÁ': 'CE',
}

def padronizar_estado(valor):
    if pd.isna(valor):
        return 'Não informado'
    texto = str(valor).strip().upper()
    match = re.search(r'\(([A-Z]{2})\)', texto)
    if match:
        sigla = match.group(1)
        return sigla if sigla in SIGLAS_BR else 'Exterior'
    if texto in SIGLAS_BR:
        return texto
    return NOMES_PARA_SIGLA.get(texto, 'Exterior')

df['estado_limpo'] = df['estado'].apply(padronizar_estado)

# Ano de fundação: normaliza números, textos e objetos datetime
# Um respondente digitou a data completa (10/06/2021) no lugar do ano,
# gerando o valor 1062021 — corrigido com o filtro de intervalo válido
def limpar_ano(valor):
    if pd.isna(valor):
        return None
    if isinstance(valor, (int, float)):
        return int(valor)
    if hasattr(valor, 'year'):
        return valor.year
    try:
        return int(str(valor)[:4])
    except:
        return None

df['ano_fundacao'] = df['ano_fundacao'].apply(limpar_ano)
df['ano_fundacao'] = df['ano_fundacao'].where(df['ano_fundacao'].between(1900, 2025), other=None)

# Sinaliza outliers de faturamento acima de R$ 1 bilhão para contexto nos gráficos
# A Telar Engenharia (R$ 11,3 bi) foi verificada e confirmada como dado legítimo
df['faturamento_outlier'] = df['faturamento_num'] > 1_000_000_000

# Padroniza nomes de cidades com variações recorrentes na base
# O caractere \xa0 (espaço não-separável) causava falha no merge com o shapefile
CORRECOES_CIDADES = {
    'SAO PAULO':      'SÃO PAULO',
    'SAO CARLOS':     'SÃO CARLOS',
    'RIBEIRAO PRETO': 'RIBEIRÃO PRETO',
}

def padronizar_cidade(cidade):
    if pd.isna(cidade):
        return cidade
    cidade = str(cidade).replace('\xa0', ' ').strip()
    return CORRECOES_CIDADES.get(cidade.upper(), cidade)

df['cidade'] = df['cidade'].apply(padronizar_cidade)

print(f"Total de empresas:                {len(df)}")
print(f"Sem faturamento informado:        {df['faturamento_num'].isna().sum()}")
print(f"Empresas do exterior:             {(df['estado_limpo'] == 'Exterior').sum()}")
print(f"Outliers de faturamento (>1 bi):  {df['faturamento_outlier'].sum()}")
print(f"Registros com ano invalido:       {df['ano_fundacao'].isna().sum()}")

## 2. Visão Geral da Base

Esta seção apresenta a distribuição das empresas por faturamento, ano de fundação e localização geográfica.

### Distribuição de Faturamento

Um boxplot resume a distribuição de valores em cinco medidas: mínimo, máximo, mediana e os quartis Q1 (25%) e Q3 (75%). Pontos fora dos "bigodes" são outliers — valores muito distantes do intervalo esperado.

O gráfico é apresentado em dois painéis. O primeiro mostra a escala real, evidenciando o impacto dos valores extremos. O segundo remove o top 5% da distribuição para revelar com mais clareza como a maioria das empresas se distribui. Empresas sem faturamento declarado foram excluídas desta visualização.

In [ ]:
df_fat = df.dropna(subset=['faturamento_num']).copy()
p95    = df_fat['faturamento_num'].quantile(0.95)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Distribuição de Faturamento', fontsize=16, fontweight='bold')

sns.boxplot(y=df_fat['faturamento_num'], ax=axes[0], color='#2d6a4f')
axes[0].set_title('Com outliers')
axes[0].set_ylabel('Faturamento')
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'R$ {x/1e6:.0f}M'))

sns.boxplot(y=df_fat[df_fat['faturamento_num'] <= p95]['faturamento_num'], ax=axes[1], color='#2d6a4f')
axes[1].set_title(f'Sem top 5% (até R$ {p95/1e3:.0f}k)')
axes[1].set_ylabel('')
axes[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'R$ {x/1e3:.0f}k'))

plt.tight_layout()
plt.show()

# Lista das empresas acima do percentil 95 para verificação manual
outliers = (df_fat[df_fat['faturamento_num'] > p95]
            [['nome', 'cidade', 'estado_limpo', 'faturamento_num']]
            .sort_values('faturamento_num', ascending=False)
            .assign(faturamento=lambda x: x['faturamento_num']
                    .apply(lambda v: f'R$ {v:,.0f}'.replace(',', '.')))
            .drop(columns='faturamento_num'))

print(f"Empresas acima do percentil 95 (R$ {p95/1e3:.0f}k): {len(outliers)}")
print(outliers.to_string(index=False))

### Empresas Fundadas por Ano

O gráfico apresenta a evolução histórica de abertura de empresas cadastradas, a partir de 1963. A linha laranja representa a média móvel de 3 anos, que suaviza variações pontuais e torna a tendência de longo prazo mais legível. O crescimento acelerado a partir dos anos 2000 reflete o amadurecimento do ecossistema empreendedor brasileiro.

In [ ]:
df_ano = (df[df['ano_fundacao'] >= 1963]
            .dropna(subset=['ano_fundacao'])
            .groupby('ano_fundacao')
            .size()
            .reset_index(name='quantidade'))

df_ano['media_movel'] = df_ano['quantidade'].rolling(window=3, center=True).mean()

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(df_ano['ano_fundacao'], df_ano['quantidade'],
       color='#2d6a4f', alpha=0.85, width=0.8)
ax.plot(df_ano['ano_fundacao'], df_ano['media_movel'],
        color='#f4a261', linewidth=2.5, label='Média móvel (3 anos)')
ax.set_title('Empresas Fundadas por Ano', fontsize=14, fontweight='bold')
ax.set_xlabel('Ano de Fundação')
ax.set_ylabel('Quantidade de Empresas')
ax.xaxis.set_major_locator(mticker.MultipleLocator(2))
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Distribuição Geográfica por Estado

O mapa apresenta a concentração de empresas por estado, utilizando escala de cores logarítmica. Essa escolha é necessária porque São Paulo concentra 87% da base, o que tornaria os demais estados praticamente invisíveis em uma escala linear. Com a escala logarítmica, as diferenças entre estados menores ficam visíveis sem perder a noção de que SP domina amplamente a distribuição. Estados sem nenhuma empresa cadastrada aparecem em cinza claro.

In [ ]:
brasil = gpd.read_file(
    'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson'
)
brasil['sigla'] = brasil['sigla'].str.upper()

df_estado = df.groupby('estado_limpo').size().reset_index(name='quantidade')
mapa      = brasil.merge(df_estado, left_on='sigla', right_on='estado_limpo', how='left')
mapa['quantidade'] = mapa['quantidade'].fillna(0)

fig, ax = plt.subplots(figsize=(10, 12))
norm = LogNorm(vmin=1, vmax=mapa['quantidade'].max())

mapa[mapa['quantidade'] > 0].plot(
    column='quantidade', cmap='YlOrRd', linewidth=0.8,
    edgecolor='white', legend=False, norm=norm, ax=ax)
mapa[mapa['quantidade'] == 0].plot(
    color='#f0f0f0', edgecolor='white', linewidth=0.8, ax=ax)

sm = plt.cm.ScalarMappable(cmap='YlOrRd', norm=norm)
sm.set_array([])
fig.colorbar(sm, ax=ax, shrink=0.5, label='Nº de Empresas (escala log)')

for _, row in mapa.iterrows():
    centroid = row.geometry.centroid
    qtd   = int(row['quantidade'])
    label = f"{row['sigla']}\n{qtd}" if qtd > 0 else row['sigla']
    ax.annotate(label, xy=(centroid.x, centroid.y),
                ha='center', fontsize=7, color='#333333')

ax.set_title('Distribuição de Empresas por Estado', fontsize=14, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.show()

### Mapa Interativo de Faturamento por Município — SP

O mapa interativo permite explorar o faturamento médio por município do estado de São Paulo. A coloração segue escala logarítmica pelo mesmo motivo do mapa estadual: a capital concentra a maior parte das empresas e puxaria toda a escala para si em uma representação linear.

Para visualizar, passe o mouse sobre qualquer município. O tooltip exibe nome, faturamento médio e número de empresas cadastradas. Municípios sem dados aparecem em cinza claro. O arquivo também é salvo em `mapa_sp_faturamento.html` para compartilhamento.

In [ ]:
# Carrega o shapefile dos municípios de SP e prepara os dados de faturamento
sp_mapa = gpd.read_file(
    'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-35-mun.json'
)
sp_mapa['name'] = sp_mapa['name'].str.upper().str.strip()

df_sp = df[(df['estado_limpo'] == 'SP') & df['faturamento_num'].notna()].copy()

# Correções de nomes de municípios para garantir o merge com o shapefile
CORRECOES_SP = {
    'SAO PAULO':             'SÃO PAULO',
    'SAO CARLOS':            'SÃO CARLOS',
    'EMBU DAS ARTES':        'EMBU',
    "SANTA BARBARA D'OESTE": "SANTA BÁRBARA D'OESTE",
    'SANTO AMARO':           'SÃO PAULO',
}

def normalizar_cidade_sp(nome):
    if pd.isna(nome):
        return None
    nome = str(nome).replace('\xa0', ' ').strip().replace('-SP', '').strip().replace('´', "'")
    return CORRECOES_SP.get(nome, nome)

df_sp['cidade'] = df_sp['cidade'].apply(normalizar_cidade_sp)
df_sp = df_sp[df_sp['cidade'] != 'N/D']

df_sp_cidade = (df_sp.groupby('cidade')['faturamento_num']
                     .agg(media='mean', count='count')
                     .reset_index())

mapa_sp = sp_mapa.merge(df_sp_cidade, left_on='name', right_on='cidade', how='left')
mapa_sp['media']     = mapa_sp['media'].fillna(0)
mapa_sp['count']     = mapa_sp['count'].fillna(0).astype(int)
mapa_sp['media_log'] = mapa_sp['media'].apply(lambda x: np.log10(x) if x > 0 else 0)
mapa_sp['media_fmt'] = mapa_sp['media'].apply(
    lambda x: f'R$ {x:,.0f}'.replace(',', '.') if x > 0 else 'Sem dados')

m = folium.Map(location=[-22.5, -48.5], zoom_start=7, tiles='CartoDB positron')

folium.Choropleth(
    geo_data=mapa_sp.__geo_interface__,
    data=mapa_sp,
    columns=['name', 'media_log'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=0.3,
    legend_name='Faturamento Médio — escala log10 (R$)',
    nan_fill_color='#f0f0f0',
).add_to(m)

folium.GeoJson(
    mapa_sp.__geo_interface__,
    style_function=lambda x: {'fillOpacity': 0, 'weight': 0},
    tooltip=GeoJsonTooltip(
        fields=['name', 'media_fmt', 'count'],
        aliases=['Município', 'Faturamento Médio', 'Nº de Empresas'],
        localize=True, sticky=True)
).add_to(m)

m.save('mapa_sp_faturamento.html')
print(f"Mapa salvo. Municípios com dados: {(mapa_sp['media'] > 0).sum()} de {len(mapa_sp)}")

## 3. Recorte: Agronegócio

Para identificar empresas do agronegócio na base, foram combinados dois critérios:

**CNAEs Core** — divisões diretamente ligadas ao setor, incluídas integralmente: `01` Agricultura e Pecuária, `02` Produção Florestal, `03` Pesca e Aquicultura, `10` Fabricação de Alimentos, `20` Defensivos e Fertilizantes, `28` Máquinas Agrícolas e `75` Atividades Veterinárias.

**CNAEs filtrados por descrição** — as divisões `72` (P&D) e `46` (Comércio Atacadista) são mais amplas e incluem empresas de outros setores. Por isso, foram incluídas somente quando a descrição da empresa menciona termos relacionados ao agronegócio, como `agro`, `agricultura`, `biotecnologia`, `insumos`, entre outros.

Esse método resultou em **227 empresas**, divididas em 157 do Core Agro e 70 de Agtech/Biotech.

In [ ]:
KEYWORDS_AGRO = [
    'agro', 'agrícol', 'agricultur', 'pecuári', 'bovino', 'suíno', 'avícol',
    'rural', 'fazenda', 'lavoura', 'colheit', 'plantio', 'solo', 'fertil',
    'defensivo', 'insumo', 'semente', 'irrigaç', 'biológico', 'bioestimulante',
    'inoculante', 'fitopatolog', 'fitossanit', 'biotech', 'biotecnolog',
    'veterinári', 'zootecni', 'aquicultur', 'florestal', 'silvicultur',
    'agtech', 'precisão', 'drone.*agro', 'microalga', 'biofertil',
    'controle biológico', 'microrganismo', 'microbiota', 'genômica',
    'alimento.*agro', 'cadeia produtiva.*agro'
]

def contem_keyword(texto):
    if pd.isna(texto):
        return False
    return any(re.search(kw, texto.lower()) for kw in KEYWORDS_AGRO)

CNAE_CORE     = ['01', '02', '03', '10', '20', '28', '75']
CNAE_FILTRADO = ['72', '46']

df_agro = df[
    df['cnae_divisao'].isin(CNAE_CORE) |
    (df['cnae_divisao'].isin(CNAE_FILTRADO) & df['descricao'].apply(contem_keyword))
].copy()

df_agro['origem'] = df_agro['cnae_divisao'].apply(
    lambda x: 'Core Agro' if x in CNAE_CORE else 'Agtech/Biotech')

### Nuvem de Palavras — Agronegócio

Gerada a partir das descrições das 227 empresas do recorte agro, após remoção de stopwords (artigos, preposições e termos genéricos). O tamanho de cada palavra é proporcional à sua frequência nas descrições. As palavras `agricultura` e `agrícola` foram unificadas em `agro` para evitar duplicidade visual.

In [ ]:
STOPWORDS_AGRO = set(STOPWORDS)
STOPWORDS_AGRO.update([
    'ao', 'aos', 'da', 'das', 'de', 'do', 'dos', 'du', 'na', 'nas', 'no', 'nos', 'num', 'numa',
    'um', 'uns', 'uma', 'umas', 'se', 'em', 'é', 'ou', 'às', 'à', 'a', 'o', 'os',
    'pela', 'pelo', 'pelas', 'pelos', 'por', 'para', 'com', 'que', 'como',
    'entre', 'além', 'através', 'ainda', 'sendo', 'também', 'mais', 'bem', 'até',
    'sua', 'suas', 'seu', 'seus', 'nosso', 'nossa', 'nossos', 'nossas',
    'não', 'há', 'todo', 'toda', 'todos', 'todas', 'cada', 'esta', 'este',
    'essa', 'esse', 'isso', 'isto', 'aqui', 'ser', 'estar', 'visando',
    'ano', 'anos', 'assim', 'conta', 'pessoas', 'vida', 'maior', 'alto', 'd', 'e',
    'foi', 'tem', 'são', 'atua', 'atuamos', 'oferece', 'oferecemos',
    'desenvolve', 'desenvolvemos', 'busca', 'somos', 'pode', 'podem',
    'empresa', 'empresas', 'serviço', 'serviços', 'área', 'áreas',
    'solução', 'soluções', 'mercado', 'tecnologia', 'inovação',
    'desenvolvimento', 'produtos', 'produto', 'base', 'brasil',
    'brasileira', 'brasileiras', 'brasileiros', 'missão', 'necessidade',
    'objetivo', 'forma', 'grande', 'resultado', 'resultados', 'custo', 'uso',
    'novo', 'nova', 'novos', 'novas', 'sempre', 'desde', 'foco',
    'fundada', 'fundado', 'outro', 'outros', 'outra', 'outras', 'onde',
    'parceria', 'parcerias', 'voltado', 'voltada', 'focado', 'focada',
    'especializado', 'especializada', 'inovador', 'inovadora',
    'veterinário', 'veterinária', 'veterinários', 'veterinárias',
    'paulo', 'são', 'usp', 'clínico', 'clínica', 'paciente', 'pacientes',
    'cães', 'gato', 'gatos', 'pet', 'medicina', 'médico', 'médica',
    'diagnóstico', 'atendimento', 'cliente', 'clientes', 'startup', 'startups',
    'meio', 'equipe', 'setor', 'campo', 'alta', 'sobre', 'diversas', 'diversos',
    'atividade', 'atividades', 'exame', 'exames', 'cursos', 'curso', 'ciência',
    'conhecimento', 'experiência', 'excelência', 'tecnológica', 'tecnológico',
    'plataforma', 'ingrediente', 'ingredientes', 'técnica', 'técnico',
    'insumo', 'baseada', 'sem', 'está', 'possui', 'sistema',
])

texto_agro = ' '.join(df_agro['descricao'].dropna().tolist()).lower()
texto_agro = texto_agro.replace('agricultura', 'agro').replace('agrícola', 'agro')

wc = WordCloud(
    width=1400, height=700, background_color='white', colormap='YlGn',
    stopwords=STOPWORDS_AGRO, max_words=50, collocations=False,
    prefer_horizontal=1.0, min_font_size=12, max_font_size=120,
).generate(texto_agro)

fig, ax = plt.subplots(figsize=(14, 7))
ax.imshow(wc, interpolation='bilinear')
ax.axis('off')
ax.set_title('Palavras mais frequentes — Empresas do Agronegócio',
             fontsize=14, fontweight='bold', pad=15)
plt.tight_layout()
plt.show()

## 4. Recorte: Ribeirão Preto

Ribeirão Preto concentra 267 empresas na base — 8,3% do total — sendo o segundo maior polo do estado após São Paulo. As análises a seguir exploram o perfil setorial, o faturamento médio por setor e as empresas do agronegócio presentes na cidade.

Para este recorte, foi necessário criar um dicionário de nomes legíveis para os códigos CNAE, facilitando a leitura dos gráficos.

In [ ]:
# Padroniza o nome de Ribeirão Preto, que aparecia com espaços especiais e variações de acentuação
def corrigir_ribeirao(cidade):
    if pd.isna(cidade):
        return cidade
    cidade_norm = str(cidade).strip().upper().replace('\xa0', ' ')
    if 'RIBEIRAO PRETO' in cidade_norm or 'RIBEIRÃO PRETO' in cidade_norm:
        return 'RIBEIRÃO PRETO'
    return cidade.strip()

df['cidade'] = df['cidade'].apply(corrigir_ribeirao)
df_rp = df[df['cidade'] == 'RIBEIRÃO PRETO'].copy()

# Mapa completo de nomes legíveis por divisão CNAE
CNAE_NOMES = {
    '01': 'Agricultura e Pecuária',    '02': 'Produção Florestal',
    '03': 'Pesca e Aquicultura',       '10': 'Fabricação de Alimentos',
    '20': 'Defensivos e Fertilizantes','21': 'Farmacêutica',
    '26': 'Eletrônicos e Informática', '27': 'Equipamentos Elétricos',
    '28': 'Máquinas Agrícolas',        '32': 'Fabricação Diversa',
    '33': 'Manutenção Industrial',     '35': 'Energia',
    '41': 'Construção Civil',          '43': 'Construção Civil',
    '46': 'Comércio Atacadista',       '47': 'Comércio Varejista',
    '56': 'Alimentação e Restaurantes','58': 'Edição e Publicação',
    '59': 'Cinema e Audiovisual',      '62': 'Tecnologia da Informação',
    '63': 'Serviços de TI e Informação','64': 'Serviços Financeiros',
    '66': 'Seguros e Finanças',        '68': 'Imobiliário',
    '69': 'Atividades Jurídicas',      '70': 'Consultoria e Gestão',
    '71': 'Engenharia e P&D',          '72': 'Pesquisa e Desenvolvimento',
    '73': 'Publicidade',               '74': 'Design e Fotografia',
    '75': 'Atividades Veterinárias',   '79': 'Agências de Viagem',
    '81': 'Serviços de Limpeza',       '82': 'Serviços Administrativos',
    '85': 'Educação',                  '86': 'Saúde',
    '87': 'Assistência Social',        '90': 'Artes e Cultura',
    '93': 'Esporte e Lazer',           '94': 'Organizações e Associações',
    '95': 'Reparação de Equipamentos', '96': 'Outros Serviços Pessoais',
}

df['cnae_nome']    = df['cnae_divisao'].map(CNAE_NOMES).fillna('Outros')
df_rp['cnae_nome'] = df_rp['cnae_divisao'].map(CNAE_NOMES).fillna('Outros')
df_agro['cnae_nome'] = df_agro['cnae_divisao'].map(CNAE_NOMES).fillna('Outros')

### Distribuição por Setor

Distribuição das 267 empresas de Ribeirão Preto por divisão de atividade econômica. Os setores de Saúde, Educação e Tecnologia da Informação lideram, refletindo o perfil universitário e de serviços da cidade.

In [ ]:
df_rp_cnae = (df_rp.groupby('cnae_nome')
                   .size()
                   .reset_index(name='quantidade')
                   .sort_values('quantidade', ascending=True))

fig, ax = plt.subplots(figsize=(10, 8))
bars = ax.barh(df_rp_cnae['cnae_nome'], df_rp_cnae['quantidade'],
               color='#2d6a4f', alpha=0.85)

for bar, val in zip(bars, df_rp_cnae['quantidade']):
    ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height() / 2,
            str(val), va='center', fontsize=9)

ax.set_title('Distribuição por Setor — Ribeirão Preto', fontsize=14, fontweight='bold')
ax.set_xlabel('Nº de Empresas')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

### Faturamento Médio por Setor

Faturamento médio por setor, considerando apenas setores com ao menos 3 empresas com dado informado. A escala logarítmica foi adotada porque o setor de Tecnologia da Informação apresenta faturamento médio muito acima dos demais, o que comprimiria todos os outros valores em uma escala linear.

In [ ]:
df_rp_fat = (df_rp.dropna(subset=['faturamento_num'])
                  .groupby('cnae_nome')['faturamento_num']
                  .agg(media='mean', count='count')
                  .query('count >= 3')
                  .reset_index()
                  .sort_values('media', ascending=True))

fig, ax = plt.subplots(figsize=(10, 7))
bars = ax.barh(df_rp_fat['cnae_nome'], df_rp_fat['media'],
               color='#40916c', alpha=0.85)

for bar, val in zip(bars, df_rp_fat['media']):
    label = f'R$ {val/1e3:.0f}k' if val < 1e6 else f'R$ {val/1e6:.1f}M'
    ax.text(bar.get_width() * 0.95, bar.get_y() + bar.get_height() / 2,
            label, va='center', ha='right', fontsize=9, color='white', fontweight='bold')

ax.set_xscale('log')
ax.xaxis.set_major_formatter(mticker.FuncFormatter(
    lambda x, _: f'R$ {x/1e3:.0f}k' if x < 1e6 else f'R$ {x/1e6:.1f}M'))
ax.set_title('Faturamento Médio por Setor — Ribeirão Preto\n(escala logarítmica, mín. 3 empresas)',
             fontsize=14, fontweight='bold')
ax.set_xlabel('Faturamento Médio (escala log)')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

### Empresas do Agronegócio em Ribeirão Preto

Das 267 empresas de Ribeirão Preto, 22 pertencem ao recorte do agronegócio — representando 8,2% do total local. O gráfico mostra a distribuição por subsetor, e a listagem completa é exibida abaixo.

In [ ]:
df_rp_agro = df_rp[df_rp.index.isin(df_agro.index)].copy()

df_rp_agro_cnae = (df_rp_agro.groupby('cnae_nome')
                              .size()
                              .reset_index(name='quantidade')
                              .sort_values('quantidade', ascending=True))

fig, ax = plt.subplots(figsize=(8, 5))
bars = ax.barh(df_rp_agro_cnae['cnae_nome'], df_rp_agro_cnae['quantidade'],
               color='#f4a261', alpha=0.85)

for bar, val in zip(bars, df_rp_agro_cnae['quantidade']):
    ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height() / 2,
            str(val), va='center', fontsize=9)

ax.set_title(f'Empresas do Agronegócio — Ribeirão Preto ({len(df_rp_agro)} empresas)',
             fontsize=14, fontweight='bold')
ax.set_xlabel('Nº de Empresas')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

print(f"Empresas do agronegócio em Ribeirão Preto ({len(df_rp_agro)}):\n")
print(df_rp_agro[['nome', 'cnae_nome', 'faturamento_num']]
      .sort_values('cnae_nome')
      .assign(faturamento=lambda x: x['faturamento_num']
              .apply(lambda v: f'R$ {v:,.0f}'.replace(',', '.') if pd.notna(v) else 'Não informado'))
      .drop(columns='faturamento_num')
      .to_string(index=False))

## 5. Recorte: Pesquisa e Desenvolvimento (CNAE 72)

Esta seção analisa as 157 empresas classificadas no CNAE 72, que compreende atividades de pesquisa e desenvolvimento científico. Esse grupo é diretamente relevante para o ecossistema de inovação, patentes e propriedade intelectual.

A análise cobre três perspectivas: evolução temporal da abertura dessas empresas, concentração geográfica e o vocabulário predominante nas descrições de negócio.

In [ ]:
# Correção adicional de nomes de cidades no recorte de P&D
# São Paulo e São Carlos apareciam duplicados por variação de acentuação
CORRECOES_PD = {'SAO PAULO': 'SÃO PAULO', 'SAO CARLOS': 'SÃO CARLOS'}
df['cidade'] = df['cidade'].apply(
    lambda x: CORRECOES_PD.get(str(x).strip().upper(), x) if pd.notna(x) else x)

df_pd = df[df['cnae_divisao'] == '72'].copy()

### Crescimento por Ano de Fundação

O gráfico mostra a evolução anual de abertura de empresas de P&D desde 1990. A linha laranja representa a média móvel de 3 anos. O crescimento consistente a partir de 2002 e o pico em 2021 refletem o amadurecimento do ecossistema de inovação e o impulso dado pela pandemia a investimentos em biotecnologia, saúde e agtech. A queda aparente em 2024-2025 é esperada, pois empresas recém-fundadas ainda podem não ter ingressado no programa.

In [ ]:
df_pd_ano = (df_pd[df_pd['ano_fundacao'] >= 1990]
               .groupby('ano_fundacao')
               .size()
               .reset_index(name='quantidade'))

df_pd_ano['media_movel'] = df_pd_ano['quantidade'].rolling(window=3, center=True).mean()

fig, ax = plt.subplots(figsize=(14, 5))
ax.bar(df_pd_ano['ano_fundacao'], df_pd_ano['quantidade'],
       color='#2d6a4f', alpha=0.75, width=0.8, label='Empresas fundadas')
ax.plot(df_pd_ano['ano_fundacao'], df_pd_ano['media_movel'],
        color='#f4a261', linewidth=2.5, label='Média móvel (3 anos)')
ax.set_title('Crescimento de Empresas de P&D por Ano de Fundação',
             fontsize=14, fontweight='bold')
ax.set_xlabel('Ano de Fundação')
ax.set_ylabel('Quantidade de Empresas')
ax.xaxis.set_major_locator(mticker.MultipleLocator(2))
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Concentração por Cidade

Ribeirão Preto é o segundo maior polo de P&D do estado, com 28 empresas — mais do dobro de São Carlos (12) e Piracicaba (11), tradicionais cidades universitárias do interior paulista. Esse dado reforça o peso do ecossistema de inovação da cidade, ancorado em instituições como USP e UNESP e no AgTech Valley.

In [ ]:
df_pd_cidade = (df_pd.groupby('cidade')
                     .size()
                     .reset_index(name='quantidade')
                     .sort_values('quantidade', ascending=True)
                     .tail(12))

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(df_pd_cidade['cidade'], df_pd_cidade['quantidade'],
               color='#2d6a4f', alpha=0.85)

for bar, val in zip(bars, df_pd_cidade['quantidade']):
    ax.text(bar.get_width() * 0.95, bar.get_y() + bar.get_height() / 2,
            str(val), va='center', ha='right', fontsize=10,
            color='white', fontweight='bold')

ax.set_title('Concentração de Empresas de P&D por Cidade',
             fontsize=14, fontweight='bold')
ax.set_xlabel('Nº de Empresas')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

### Vocabulário de Inovação

Nuvem de palavras gerada a partir das descrições das 157 empresas de P&D, após remoção de stopwords. O tamanho de cada palavra é proporcional à sua frequência. O vocabulário evidencia dois eixos temáticos principais: ciências da vida (biotecnologia, diagnóstico, saúde, proteína, microrganismos) e agronegócio (agrícola, agricultura, insumos, sustentável), confirmando a intersecção entre inovação e agro que caracteriza o ecossistema mapeado.

In [ ]:
STOPWORDS_PD = set(STOPWORDS)
STOPWORDS_PD.update([
    'ao', 'aos', 'da', 'das', 'de', 'do', 'dos', 'na', 'nas', 'no', 'nos',
    'um', 'uma', 'uns', 'umas', 'se', 'em', 'é', 'ou', 'à', 'às', 'a', 'o', 'os',
    'pela', 'pelo', 'por', 'para', 'com', 'que', 'como', 'entre', 'além',
    'através', 'ainda', 'sendo', 'também', 'mais', 'bem', 'até', 'd', 'e',
    'sua', 'suas', 'seu', 'seus', 'nosso', 'nossa', 'nossos', 'nossas',
    'não', 'há', 'todo', 'toda', 'todos', 'todas', 'cada', 'esta', 'este',
    'esse', 'essa', 'isso', 'ser', 'estar', 'visando', 'ano', 'assim',
    'conta', 'pessoas', 'vida', 'maior', 'alto',
    'foi', 'tem', 'são', 'atua', 'atuamos', 'oferece', 'desenvolve', 'somos',
    'pode', 'podem', 'busca', 'fundada', 'fundado',
    'empresa', 'empresas', 'serviço', 'serviços', 'área', 'áreas',
    'solução', 'soluções', 'mercado', 'tecnologia', 'inovação',
    'desenvolvimento', 'produtos', 'produto', 'base', 'anos', 'brasil',
    'brasileira', 'missão', 'necessidade', 'objetivo', 'forma', 'grande',
    'resultado', 'novo', 'nova', 'sempre', 'desde', 'foco',
    'outro', 'outros', 'outra', 'outras', 'onde', 'parceria',
    'voltada', 'voltado', 'focada', 'focado', 'especializada', 'especializado',
    'inovadora', 'paulo', 'são', 'usp',
    'dado', 'dados', 'meio', 'equipe', 'está', 'sobre', 'uso',
    'técnica', 'nacional', 'humana', 'social', 'trabalho', 'setor',
    'negócio', 'programa', 'profissionais', 'startup', 'startups',
    'cliente', 'clientes',
])

texto_pd = ' '.join(df_pd['descricao'].dropna().tolist()).lower()

wc = WordCloud(
    width=1400, height=700, background_color='white', colormap='YlGn',
    stopwords=STOPWORDS_PD, max_words=60, collocations=False,
    prefer_horizontal=1.0, min_font_size=12, max_font_size=120,
).generate(texto_pd)

fig, ax = plt.subplots(figsize=(14, 7))
ax.imshow(wc, interpolation='bilinear')
ax.axis('off')
ax.set_title('Vocabulário de Inovação — Empresas de P&D',
             fontsize=14, fontweight='bold', pad=15)
plt.tight_layout()
plt.show()

## 6. Comparativo: Agtech/Biotech vs. Agro Tradicional

Esta seção compara os dois grupos identificados no recorte do agronegócio: empresas do Core Agro (atividades primárias e industriais diretamente ligadas ao setor) e empresas de Agtech/Biotech (identificadas pelo cruzamento de CNAE de P&D e Comércio com termos de inovação nas descrições).

O painel da esquerda mostra a distribuição em quantidade de empresas. O painel da direita apresenta o faturamento médio excluindo o top 5% para evitar distorção por outliers.

In [ ]:
p95_agro   = df_agro['faturamento_num'].quantile(0.95)
df_agro_fat = (df_agro[df_agro['faturamento_num'] <= p95_agro]
               .dropna(subset=['faturamento_num'])
               .groupby('origem')['faturamento_num']
               .mean()
               .reset_index())

contagem = df_agro['origem'].value_counts().reset_index()
contagem.columns = ['origem', 'quantidade']

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Agtech/Biotech vs. Agro Tradicional', fontsize=16, fontweight='bold')

# Painel 1: quantidade de empresas
axes[0].bar(contagem['origem'], contagem['quantidade'],
            color=['#2d6a4f', '#74c69d'], alpha=0.85, width=0.5)
for i, (_, row) in enumerate(contagem.iterrows()):
    axes[0].text(i, row['quantidade'] + 2, str(row['quantidade']),
                 ha='center', fontsize=11, fontweight='bold')
axes[0].set_title('Quantidade de Empresas')
axes[0].set_ylabel('Nº de Empresas')
axes[0].set_ylim(0, contagem['quantidade'].max() * 1.15)

# Painel 2: faturamento médio sem top 5%
axes[1].bar(df_agro_fat['origem'], df_agro_fat['faturamento_num'],
            color=['#2d6a4f', '#74c69d'], alpha=0.85, width=0.5)
for i, (_, row) in enumerate(df_agro_fat.iterrows()):
    label = f"R$ {row['faturamento_num']/1e3:.0f}k"
    axes[1].text(i, row['faturamento_num'] * 1.03, label,
                 ha='center', fontsize=10, fontweight='bold')
axes[1].set_title('Faturamento Médio (sem top 5%)')
axes[1].set_ylabel('Faturamento Médio (R$)')
axes[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'R$ {x/1e3:.0f}k'))
axes[1].set_ylim(0, df_agro_fat['faturamento_num'].max() * 1.2)

plt.tight_layout()
plt.show()